# 20 뉴스 그룹 분류

다시한번 쉬운거 하면서 자신감을 가져보자 

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import fetch_20newsgroups
news = fetch_20newsgroups(subset='all', random_state=2022)

- 데이터 탐색

In [3]:
# print(news.DESCR())

In [4]:
print(news.target_names)

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


In [5]:
np.unique(news.target, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19]),
 array([799, 973, 985, 982, 963, 988, 975, 990, 996, 994, 999, 991, 984,
        990, 987, 997, 910, 940, 775, 628], dtype=int64))

In [6]:
# print(news.data[0])
print(news.data[2])

From: astein@nysernet.org (Alan Stein)
Subject: Re: Hamza Salah, the Humanist
Organization: NYSERNet, Inc.
Lines: 16

dzk@cs.brown.edu (Danny Keren) writes:

>cl056@cleveland.Freenet.Edu (Hamaza H. Salah) writes:

># Well said Mr. Beyer :)

>He-he. The great humanist speaks. One has to read Mr. Salah's posters,
>in which he decribes Jews as "sons of pigs and monkeys", keeps
>promising the "final battle" between Muslims and Jews (in which the
>stons and the trees will "cry for the Muslims to come and kill the
>Jews hiding behind them"), makes jokes about Jews dying from heart
>attacks etc, to realize his objective stance on the matters involved.

Humanist, or sub-humanist? :-)
-- 
Alan H. Stein                     astein@israel.nysernet.org



In [7]:
# 예뻐예뻐 머리부터발끝까지 헤더와 푸터 header, footer 

In [8]:
# 한번 불러오면 그다음은 빠른가 ? 헤더와 쿠토스? 푸터 생략하고 맞아오기 

In [9]:
train_news = fetch_20newsgroups(
    subset='train', random_state=2022, remove=('header', 'quotes', 'footers')
)
X_train = train_news.data
y_train = train_news.target

In [10]:
print(X_train[1])

From: pcarmack@gimp.kpc.com (Phil Carmack)
Subject: Re: ATI ultra pro Drivers? [bad ATI ultra]
Organization: Kubota Pacific Computer
Lines: 39

I have the local bus card also, and don't have any such problems with it
now, but this is the second card I've gotten - the first card didn't work
in VGA mode correctly.  Maybe they still have some quality control problems.
I would suggest checking with ATI (I went through the vendor I bought the
card from since the problem showed up immediately).  I never was able to
get through to ATI's technical support number.  

I sure like the way the card performs though.  I have the 2MB ATI ultra
pro - local bus, and it is fast even in 1024x768x16bpp mode.


Cheers,
Phil




In [11]:
y_train[1], train_news.target_names[y_train[1]] 
# 요코드 뭐지 파이어뱃. 파워

(2, 'comp.os.ms-windows.misc')

In [12]:
train_news = fetch_20newsgroups(
    subset='test', random_state=2022, remove=('header', 'quotes', 'footers')
)
X_test = train_news.data
y_test = train_news.target

In [13]:
len(X_train), len(X_test)

(11314, 7532)

피쳐 벡터화 변환 및 머신러닝 모델 학습 / 평가


In [14]:
# case 1 

In [15]:
# from sklearn.feature_extraction.text import CountVectorizer
# cvect = CountVectorizer()
# cvect.fit(X_train)
# X_train_cv = cvect.transform(X_train)
# X_test_cv = cvect.transform(X_test)
# X_train_cv.shape, X_test_cv

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer()
cvect.fit(X_train)
X_train_cv = cvect.transform(X_train)
X_test_cv = cvect.transform(X_test)
X_train_cv.shape, X_test_cv.shape

((11314, 99238), (7532, 99238))

In [17]:
from sklearn.linear_model import LogisticRegression
lrc = LogisticRegression(random_state=2022)
%time lrc.fit(X_train_cv, y_train)
lrc.score(X_test_cv, y_test)

Wall time: 40.3 s


0.7458842272968667

#2. case2) TFidf VC + SVC 

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvect = TfidfVectorizer()
tvect.fit(X_train)
X_train_tv = tvect.transform(X_train)
X_test_tv = tvect.transform(X_test)
X_train_tv.shape, X_test_tv.shape

((11314, 99238), (7532, 99238))

In [19]:
from sklearn.svm import SVC
svc = SVC()
%time svc.fit(X_train_tv, y_train)
svc.score(X_test_tv, y_test)

Wall time: 3min 22s


0.7712426978226234

In [20]:
# tvect.get_params()

In [21]:
# - Pipeline/gridser

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [23]:
# 파이프라인은 리스트에 튜플형태
pipeline = Pipeline([
    ('tvect', TfidfVectorizer(stop_words='english')),
    ('lrc' , LogisticRegression(random_state=2022))
])
params = {
    'tvect__max_df': [300, 700],
    'tvect__ngram_range' : [(1,1),(1,2)],
    'lrc__C':[1,10]
}

In [26]:
grid_pipe = GridSearchCV(
    pipeline, param_grid=params, scoring='accuracy', cv=3, n_jobs=-1
)
%time grid_pipe.fit(X_train, y_train)

## ㄷㄷ 1시간을 돌렸는데 변수이름이 다르네 
#  n_jops = -1 을 안줘서 오래걸렸어. 

In [ ]:
grid_pipe.best_params_
# {'lrc__C': 10, 'tvect__max_df': 700, 'tvect__ngram_range': (1, 2)}

{'lrc__C': 10, 'tvect__max_df': 700, 'tvect__ngram_range': (1, 2)}

In [ ]:
grid_pipe.best_estimator_.score(X_test, y_test)
# 0.8061603823685608

0.8061603823685608

# 마지막에 변수이름만 바꿔서 저장. 

문서 군집화라는게 있어요 .군집화니까 이것도 비지도 학습일 겁니다.
UCI 머신러닝 레파지 토리. 